<a href="https://colab.research.google.com/github/sandali12345-sri/AIhealthdiseasepredictor.github.io/blob/main/AI_Health_Diseases_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import gradio as gr


In [ ]:
# Step 2: Load Datasets

# Diabetes Dataset
diabetes_df = pd.read_csv("/content/Diabetes_dataset.csv")

# Heart Disease Dataset
heart_df = pd.read_csv("/content/heart.csv")

# Parkinson's Dataset
parkinsons_df = pd.read_csv("/content/parkinsons.csv")

print("✅ Datasets Loaded Successfully!")
print("Diabetes Shape:", diabetes_df.shape)
print("Heart Shape:", heart_df.shape)
print("Parkinson's Shape:", parkinsons_df.shape)


✅ Datasets Loaded Successfully!
Diabetes Shape: (2761, 7)
Heart Shape: (1025, 14)
Parkinson's Shape: (195, 24)


In [ ]:
# Step 3: Preprocessing Function

def preprocess_dataset(df, target_col):
    X = df.drop(columns=[target_col], axis=1)
    y = df[target_col]

    # Fill missing values in 'Comorbidities' with a placeholder like 'None'
    if 'Comorbidities' in X.columns:
        X['Comorbidities'] = X['Comorbidities'].fillna('None')

    # Identify categorical columns with object or category dtype
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns

    # Apply one-hot encoding to categorical columns
    X = pd.get_dummies(X, columns=categorical_cols, dummy_well_formed=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return train_test_split(X_scaled, y, test_size=0.2, random_state=42), scaler

In [ ]:
# Step 5: Prediction Functions

def predict_diabetes(*args):
    # The last argument is the model name, the rest are features
    model_name = args[-1]
    data = np.array(args[:-1], dtype=float) # Collect features into a numpy array and ensure float type

    # Replace NaN values with the mean of the training data for each feature
    data[np.isnan(data)] = np.nanmean(diabetes_X_train, axis=0)[np.isnan(data)] # Calculate mean along columns

    data = data.reshape(1, -1) # Reshape for scaling

    model = diabetes_models[model_name]
    scaled = diabetes_scaler.transform(data)
    prediction = model.predict(scaled)[0]
    return "Diabetic" if prediction == 1 else "Not Diabetic"

def predict_heart(*args):
    # The last argument is the model name, the rest are features
    model_name = args[-1]
    data = np.array(args[:-1], dtype=float) # Collect features into a numpy array and ensure float type

    # Replace NaN values with the mean of the training data for each feature
    data[np.isnan(data)] = np.nanmean(heart_X_train, axis=0)[np.isnan(data)] # Calculate mean along columns

    data = data.reshape(1, -1) # Reshape for scaling

    model = heart_models[model_name]
    scaled = heart_scaler.transform(data)
    prediction = model.predict(scaled)[0]
    return "Heart Disease" if prediction == 1 else "Healthy Heart"

def predict_parkinsons(*args):
    # The last argument is the model name, the rest are features
    model_name = args[-1]
    data = np.array(args[:-1], dtype=float) # Collect features into a numpy array and ensure float type

    # Replace NaN values with the mean of the training data for each feature
    data[np.isnan(data)] = np.nanmean(parkinsons_X_train, axis=0)[np.isnan(data)] # Calculate mean along columns

    data = data.reshape(1, -1) # Reshape for scaling

    model = parkinsons_models[model_name]
    scaled = parkinsons_scaler.transform(data)
    prediction = model.predict(scaled)[0]
    return "Parkinson's Disease" if prediction == 1 else "Healthy"

In [ ]:
import gradio as gr
import numpy as np

# Dummy prediction functions (replace with your trained models)
def predict_diabetes(age, glucose, bp, insulin, bmi):
    # Example dummy logic
    if glucose > 125 or bmi > 30:
        return "⚠️ High Risk of Diabetes"
    else:
        return "✅ Low Risk of Diabetes"

def predict_heart(age, sex, cp, trestbps, chol, thalach, oldpeak):
    if chol > 240 or oldpeak > 2.5:
        return "⚠️ High Risk of Heart Disease"
    else:
        return "✅ Low Risk of Heart Disease"

def predict_parkinson(fo, fhi, flo, jitter, shimmer):
    if jitter > 0.005 or shimmer > 0.02:
        return "⚠️ Possible Parkinson's Detected"
    else:
        return "✅ No Parkinson's Detected"

# ----------- Diabetes Tab -----------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧑‍⚕️ AI Health Disease Predictor")
    gr.Markdown("Predict **Diabetes**, **Heart Disease**, and **Parkinson's Disease** using ML Models.")

    with gr.Tabs():
        with gr.Tab("Diabetes Predictor"):
            with gr.Row():
                age = gr.Slider(10, 100, label="Age")
                glucose = gr.Number(label="Glucose Level (mg/dL)")
            with gr.Row():
                bp = gr.Number(label="Blood Pressure (mmHg)")
                insulin = gr.Number(label="Insulin Level")
            with gr.Row():
                bmi = gr.Number(label="BMI")
            diabetes_btn = gr.Button("🔍 Predict Diabetes", variant="primary")
            diabetes_output = gr.Textbox(label="Result", interactive=False)

            diabetes_btn.click(predict_diabetes,
                               [age, glucose, bp, insulin, bmi],
                               diabetes_output)

        # ----------- Heart Disease Tab -----------
        with gr.Tab("Heart Disease Predictor"):
            with gr.Row():
                age = gr.Slider(20, 100, label="Age")
                sex = gr.Dropdown(["Male", "Female"], label="Sex")
                cp = gr.Dropdown([0, 1, 2, 3], label="Chest Pain Type")
            with gr.Row():
                trestbps = gr.Number(label="Resting Blood Pressure")
                chol = gr.Number(label="Cholesterol (mg/dL)")
                thalach = gr.Number(label="Max Heart Rate Achieved")
                oldpeak = gr.Number(label="ST Depression")
            heart_btn = gr.Button("🔍 Predict Heart Disease", variant="primary")
            heart_output = gr.Textbox(label="Result", interactive=False)

            heart_btn.click(predict_heart,
                            [age, sex, cp, trestbps, chol, thalach, oldpeak],
                            heart_output)

        # ----------- Parkinson's Tab -----------
        with gr.Tab("Parkinson's Predictor"):
            with gr.Row():
                fo = gr.Number(label="Average Vocal Fundamental Frequency (Fo)")
                fhi = gr.Number(label="Maximum Vocal Fundamental Frequency (Fhi)")
                flo = gr.Number(label="Minimum Vocal Fundamental Frequency (Flo)")
            with gr.Row():
                jitter = gr.Number(label="Jitter (%)")
                shimmer = gr.Number(label="Shimmer (%)")
            park_btn = gr.Button("🔍 Predict Parkinson's", variant="primary")
            park_output = gr.Textbox(label="Result", interactive=False)

            park_btn.click(predict_parkinson,
                           [fo, fhi, flo, jitter, shimmer],
                           park_output)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4504bab5983e25e83d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import pandas as pd

# =======================
# Utility for CSV Prediction
# =======================
def process_csv(file, disease="Diabetes"):
    if file is None:
        return "⚠️ Please upload a CSV file."

    df = pd.read_csv(file.name)

    # Example dummy checks (replace with ML model later)
    if disease == "Diabetes":
        df["Prediction"] = df.apply(
            lambda row: "⚠️ Diabetic" if row.get("Glucose", 0) > 125 or row.get("BMI", 0) > 30 else "✅ Not Diabetic", axis=1
        )
    elif disease == "Heart":
        df["Prediction"] = df.apply(
            lambda row: "⚠️ Heart Disease" if row.get("Cholesterol", 0) > 240 or row.get("Oldpeak", 0) > 2.5 else "✅ Healthy", axis=1
        )
    else:  # Parkinson's
        df["Prediction"] = df.apply(
            lambda row: "⚠️ Possible Parkinson's" if row.get("Jitter", 0) > 0.005 or row.get("Shimmer", 0) > 0.02 else "✅ No Parkinson's", axis=1
        )

    return df


# ----------- Main Interface with Tabs and CSV Upload -----------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧑‍⚕️ AI Health Disease Predictor")
    gr.Markdown("Predict **Diabetes**, **Heart Disease**, and **Parkinson's Disease** using ML Models.")

    with gr.Tabs():
        # ----------- Diabetes Tab -----------
        with gr.Tab("Diabetes Predictor"):
            gr.Markdown("Enter details manually **or upload a CSV file**.")
            with gr.Row():
                with gr.Column():
                    age = gr.Slider(10, 100, label="Age")
                    glucose = gr.Number(label="Glucose Level (mg/dL)")
                    bp = gr.Number(label="Blood Pressure (mmHg)")
                    insulin = gr.Number(label="Insulin Level")
                    bmi = gr.Number(label="BMI")
                    output = gr.Textbox(label="Result", interactive=False)
                    btn = gr.Button("🔍 Predict Single Patient", variant="primary")

                with gr.Column():
                    file = gr.File(label="📂 Upload CSV", file_types=[".csv"])
                    csv_output = gr.Dataframe(label="CSV Predictions", interactive=False)
                    csv_btn = gr.Button("📊 Predict from CSV", variant="secondary")

            btn.click(lambda a, g, b, i, m:
                          "⚠️ Diabetic" if g > 125 or m > 30 else "✅ Not Diabetic",
                          [age, glucose, bp, insulin, bmi], output)

            csv_btn.click(lambda f: process_csv(f, "Diabetes"), file, csv_output)


        # ----------- Heart Disease Tab -----------
        with gr.Tab("Heart Disease Predictor"):
            gr.Markdown("Enter patient data manually **or upload a CSV file**.")
            with gr.Row():
                with gr.Column():
                    age = gr.Slider(20, 100, label="Age")
                    sex = gr.Dropdown(["Male", "Female"], label="Sex")
                    cp = gr.Dropdown([0, 1, 2, 3], label="Chest Pain Type")
                    trestbps = gr.Number(label="Resting Blood Pressure")
                    chol = gr.Number(label="Cholesterol (mg/dL)")
                    thalach = gr.Number(label="Max Heart Rate Achieved")
                    oldpeak = gr.Number(label="ST Depression")
                    output = gr.Textbox(label="Result", interactive=False)
                    btn = gr.Button("🔍 Predict Single Patient", variant="primary")

                with gr.Column():
                    file = gr.File(label="📂 Upload CSV", file_types=[".csv"])
                    csv_output = gr.Dataframe(label="CSV Predictions", interactive=False)
                    csv_btn = gr.Button("📊 Predict from CSV", variant="secondary")

            btn.click(lambda a, s, c, t, ch, th, o:
                          "⚠️ Heart Disease" if ch > 240 or o > 2.5 else "✅ Healthy Heart",
                          [age, sex, cp, trestbps, chol, thalach, oldpeak], output)

            csv_btn.click(lambda f: process_csv(f, "Heart"), file, csv_output)


        # ----------- Parkinson's Tab -----------
        with gr.Tab("Parkinson's Predictor"):
            gr.Markdown("Enter patient data manually **or upload a CSV file**.")
            with gr.Row():
                with gr.Column():
                    fo = gr.Number(label="Average Vocal Fundamental Frequency (Fo)")
                    fhi = gr.Number(label="Maximum Vocal Fundamental Frequency (Fhi)")
                    flo = gr.Number(label="Minimum Vocal Fundamental Frequency (Flo)")
                    jitter = gr.Number(label="Jitter (%)")
                    shimmer = gr.Number(label="Shimmer (%)")
                    output = gr.Textbox(label="Result", interactive=False)
                    btn = gr.Button("🔍 Predict Single Patient", variant="primary")

                with gr.Column():
                    file = gr.File(label="📂 Upload CSV", file_types=[".csv"])
                    csv_output = gr.Dataframe(label="CSV Predictions", interactive=False)
                    csv_btn = gr.Button("📊 Predict from CSV", variant="secondary")

            btn.click(lambda fo_val, fhi_val, flo_val, jitter_val, shimmer_val:
                          "⚠️ Possible Parkinson's" if jitter_val > 0.005 or shimmer_val > 0.02 else "✅ No Parkinson's",
                          [fo, fhi, flo, jitter, shimmer], output)

            csv_btn.click(lambda f: process_csv(f, "Parkinson"), file, csv_output)


demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7062ca607b9de493ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
